<a href="https://colab.research.google.com/github/gibsonx/AnsibleGUI/blob/master/Federated_Minst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [193]:
#Modified by Gibson Xue on July 25, 2021
!pip uninstall --yes tensorflow tensorboard tb-nightly
!pip install --quiet tensorflow==2.5.0
!pip install --quiet tensorflow-federated==0.19.0
!pip install --quiet nest-asyncio
!pip install --quiet tensorboard  # or tensorboard, but not both


Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
Uninstalling tensorboard-2.5.0:
  Successfully uninstalled tensorboard-2.5.0


In [194]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import nest_asyncio
nest_asyncio.apply()
print(tf.__version__)

2.5.0


In [195]:
!git clone https://github.com/egcode/MNIST-to-CSV.git
def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()

convert("/content/MNIST-to-CSV/train-images-idx3-ubyte", "/content/MNIST-to-CSV/train-labels-idx1-ubyte",
        "mnist_train.csv", 60000)
convert("/content/MNIST-to-CSV/t10k-images-idx3-ubyte", "/content/MNIST-to-CSV/t10k-labels-idx1-ubyte",
        "mnist_test.csv", 10000)

fatal: destination path 'MNIST-to-CSV' already exists and is not an empty directory.


In [196]:
df_orig_train = pd.read_csv('mnist_train.csv')
df_orig_test = pd.read_csv('mnist_test.csv')
df_orig_train.rename(columns={'5':'label'}, inplace=True)
df_orig_test.rename(columns={'7':'label'}, inplace=True)
df_orig_train.to_csv('mnist_train_final.csv', index=False)
df_orig_test.to_csv('mnist_test_final.csv', index=False)

In [197]:
import collections
import functools
import os
import time
from tensorflow.keras.datasets import mnist
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
import pandas as pd
np.random.seed(0)

from tensorflow import reshape, nest, config
from tensorflow.keras import losses, metrics, optimizers
import tensorflow_federated as tff
from matplotlib import pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
# Test the TFF is working:
tff.federated_computation(lambda: 'Hello, World!')()

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense


def create_keras_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPool2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model

In [198]:
this_dir = Path.cwd()
model_dir = this_dir / "saved_models" / experiment_name / method
output_dir = this_dir / "results" / experiment_name / method

if not model_dir.exists():
    model_dir.mkdir(parents=True)

if not output_dir.exists():
    output_dir.mkdir(parents=True)

In [199]:
experiment_name = "mnist"
method = "tff_training"
client_lr = 1e-2
server_lr = 1e-2
split = 4
NUM_ROUNDS = 5
NUM_EPOCHS = 5
BATCH_SIZE = 20
PREFETCH_BUFFER = 10

df_orig_train = pd.read_csv('mnist_train_final.csv')
df_orig_test = pd.read_csv('mnist_test_final.csv')

X_train = df_orig_train.drop('label',axis=1).to_numpy().astype(np.int32).reshape(59999,28,28,1)
Y_train = df_orig_train['label'].to_numpy().astype(np.int32)

# x_test = df_orig_test.drop('label',axis=1).to_numpy().astype(np.int32).reshape(9999,28,28,1)
# y_test = df_orig_test['label'].to_numpy().astype(np.int32)

x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.33, random_state=42)


# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# x_train = x_train.astype(np.float32)[:300]
# y_train = y_train.astype(np.int32)[:300]
# # x_test = x_test.astype(np.float32).reshape(10000, 28, 28, 1)
# x_test = x_test.astype(np.float32)
# y_test = y_test.astype(np.int32).reshape(10000, 1)





In [200]:
x_train.shape

(40199, 28, 28, 1)

In [201]:
total_image_count = len(x_train)
image_per_set = int(np.floor(total_image_count/split))

client_train_dataset = collections.OrderedDict()
for i in range(1, split+1):
    client_name = "client_" + str(i)
    start = image_per_set * (i-1)
    end = image_per_set * i

    print(f"Adding data from {start} to {end} for client : {client_name}")
    data = collections.OrderedDict((('label', y_train[start:end]), ('pixels', x_train[start:end])))
    client_train_dataset[client_name] = data

train_dataset = tff.simulation.datasets.TestClientData(client_train_dataset)

sample_dataset = train_dataset.create_tf_dataset_for_client(train_dataset.client_ids[0])
sample_element = next(iter(sample_dataset))

SHUFFLE_BUFFER = image_per_set


def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    print(element['pixels'])
    return collections.OrderedDict(
        x=reshape(element['pixels'], [-1, 28, 28, 1]),
        y=reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

preprocessed_sample_dataset = preprocess(sample_dataset)
sample_batch = nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_sample_dataset)))

def make_federated_data(client_data, client_ids):
    return [preprocess(client_data.create_tf_dataset_for_client(x)) for x in client_ids]

federated_train_data = make_federated_data(train_dataset, train_dataset.client_ids)
print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))
federated_train_data


Adding data from 0 to 10049 for client : client_1
Adding data from 10049 to 20098 for client : client_2
Adding data from 20098 to 30147 for client : client_3
Adding data from 30147 to 40196 for client : client_4
Tensor("args_1:0", shape=(None, 28, 28, 1), dtype=int32)
Tensor("args_1:0", shape=(None, 28, 28, 1), dtype=int32)
Tensor("args_1:0", shape=(None, 28, 28, 1), dtype=int32)
Tensor("args_1:0", shape=(None, 28, 28, 1), dtype=int32)
Tensor("args_1:0", shape=(None, 28, 28, 1), dtype=int32)
Number of client datasets: 4
First dataset: <PrefetchDataset shapes: OrderedDict([(x, (None, 28, 28, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.int32), (y, tf.int32)])>


[<PrefetchDataset shapes: OrderedDict([(x, (None, 28, 28, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.int32), (y, tf.int32)])>,
 <PrefetchDataset shapes: OrderedDict([(x, (None, 28, 28, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.int32), (y, tf.int32)])>,
 <PrefetchDataset shapes: OrderedDict([(x, (None, 28, 28, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.int32), (y, tf.int32)])>,
 <PrefetchDataset shapes: OrderedDict([(x, (None, 28, 28, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.int32), (y, tf.int32)])>]

In [202]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.

  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_sample_dataset.element_spec,
      loss=losses.SparseCategoricalCrossentropy(),
      metrics=[metrics.SparseCategoricalAccuracy()])


iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: optimizers.Adam(learning_rate=client_lr),
    server_optimizer_fn=lambda: optimizers.SGD(learning_rate=server_lr))

print(str(iterative_process.initialize.type_signature))

( -> <model=<trainable=<float32[3,3,1,32],float32[32],float32[3,3,32,64],float32[64],float32[1600,512],float32[512],float32[512,10],float32[10]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)


In [203]:
state = iterative_process.initialize()

tff_train_acc = []
tff_val_acc = []
tff_train_loss = []
tff_val_loss = []

eval_model = None

In [204]:
# state.model.assign_weights_to(eval_model)
state = iterative_process.initialize()
# eval_model = create_keras_model()
# eval_model.compile(optimizer=optimizers.Adam(learning_rate=client_lr),
#                     loss=losses.SparseCategoricalCrossentropy(),
#                     metrics=[metrics.SparseCategoricalAccuracy()])

eval_model = None
for round_num in range(1, NUM_ROUNDS+1):
    state, tff_metrics = iterative_process.next(state, federated_train_data)
    eval_model = create_keras_model()
    eval_model.compile(optimizer=optimizers.Adam(learning_rate=client_lr),
                       loss=losses.SparseCategoricalCrossentropy(),
                       metrics=[metrics.SparseCategoricalAccuracy()])

    state.model.assign_weights_to(eval_model)

    ev_result = eval_model.evaluate(x_test, y_test, verbose=0)
    print('round {:2d}, metrics={}'.format(round_num, tff_metrics))
    print(f"Eval loss : {ev_result[0]} and Eval accuracy : {ev_result[1]}")


if eval_model:
    eval_model.save(model_dir / (experiment_name + ".h5"))
else:
    print("training didn't started")
    exit()                    

round  1, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.87778884), ('loss', 0.4495403)])), ('stat', OrderedDict([('num_examples', 200980)]))])
Eval loss : 16.560802459716797 and Eval accuracy : 0.14358586072921753
round  2, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.838974), ('loss', 0.54306144)])), ('stat', OrderedDict([('num_examples', 200980)]))])
Eval loss : 11.379042625427246 and Eval accuracy : 0.24424242973327637
round  3, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.8147179), ('loss', 0.6058767)])), ('stat', OrderedDict([('num_examples', 200980)]))])
Eval loss : 8.616987228393555 and Eval accuracy : 0.2810100